In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("=" * 80)
print("LINEAR REGRESSION 모델 - Life Expectancy 예측 (개선 버전)")
print("=" * 80)

# ============================================================================
# 1. 데이터 수집 및 로딩
# ============================================================================
print("\n[1단계] 데이터 수집 및 로딩")
print("-" * 80)

# CSV 파일에서 데이터 로드
# - 기대수명(Life Expectancy)과 관련된 여러 국가의 건강, 경제, 사회 지표 데이터
# - WHO에서 수집한 2000-2015년 데이터
df = pd.read_csv('dataset/LifeExpectancyData.csv')

print(f"✓ 데이터 로딩 완료")
print(f"  - 데이터 크기: {df.shape[0]}행 × {df.shape[1]}열")
print(f"  - 컬럼 수: {len(df.columns)}개")
print(f"\n컬럼 목록:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

# ============================================================================
# 2. 탐색적 데이터 분석 (EDA - Exploratory Data Analysis)
# ============================================================================
# EDA 목적: 데이터의 특성을 파악하고 문제점을 발견하기 위함
# - 결측치 확인 → 데이터 품질 평가
# - 데이터 분포 확인 → 이상치 및 편향 파악
# - 변수 간 상관관계 파악 → 다중공선성 체크
print("\n" + "=" * 80)
print("[2단계] 탐색적 데이터 분석 (EDA)")
print("=" * 80)

# 2.1 결측치 및 중복값 통계
print("\n[2.1] 결측치 및 중복값 통계")
print("-" * 80)

# 결측치(Missing Value) 확인
# - 결측치가 많으면 데이터 품질이 낮고 모델 성능에 악영향
# - 처리 방법:
#   1) 삭제: 결측치 비율이 높은 컬럼/행 제거
#   2) 대체: 평균, 중앙값, 최빈값으로 채우기
#   3) 예측: 다른 변수로 결측치 예측
missing_counts = df.isnull().sum()
missing_percent = (df.isnull().sum() / len(df) * 100).round(2)
missing_df = pd.DataFrame({
    '컬럼명': missing_counts.index,
    '결측치 개수': missing_counts.values,
    '결측치 비율(%)': missing_percent.values
})
missing_df = missing_df[missing_df['결측치 개수'] > 0].sort_values('결측치 개수', ascending=False)

print(f"전체 결측치 개수: {df.isnull().sum().sum():,}개")
print(f"중복된 행 개수: {df.duplicated().sum()}개")
print(f"\n결측치가 있는 컬럼 (상위 10개):")
if len(missing_df) > 0:
    print(missing_df.head(10).to_string(index=False))
else:
    print("  결측치 없음")

# 2.2 기초 통계량
print("\n[2.2] 기초 통계량")
print("-" * 80)
# describe(): 평균, 표준편차, 최솟값, 최댓값, 사분위수 등을 보여줌
# 용도:
# - 이상치(outlier) 확인: max가 비정상적으로 크거나 min이 음수인지 확인
# - 변수의 범위(scale) 파악: GDP(수만)와 사망률(0~1)은 스케일 차이 큼
# - 데이터 분포 추정: 평균과 중앙값(50%) 차이로 편향 정도 파악
print(df.describe().round(2))

# 2.3 주요 변수별 데이터 분포 (Histogram)
print("\n[2.3] 주요 변수별 데이터 분포 시각화")
print("-" * 80)

# 숫자형 컬럼만 선택
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Life expectancy 제외하고 주요 변수 선택 (최대 9개)
# 이유: Life expectancy는 타겟 변수이므로 독립변수 분포만 확인
plot_cols = [col for col in numeric_cols if col != 'Life expectancy '][:9]

fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.ravel()  # 2D 배열을 1D로 변환하여 인덱스 접근 쉽게

for idx, col in enumerate(plot_cols):
    # 히스토그램으로 데이터 분포 확인
    # 확인 사항:
    # - 정규분포(종 모양)인지, 편향(skewed)되어 있는지
    # - 이상치 존재 여부 (극단값이 많은지)
    # - 대부분의 값이 특정 범위에 몰려있는지
    axes[idx].hist(df[col].dropna(), bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col}', fontsize=10, fontweight='bold')
    axes[idx].set_xlabel('값', fontsize=9)
    axes[idx].set_ylabel('빈도', fontsize=9)
    axes[idx].grid(True, alpha=0.3)

# 빈 subplot 제거 (9개보다 변수가 적을 경우)
for idx in range(len(plot_cols), 9):
    fig.delaxes(axes[idx])

plt.suptitle('주요 변수별 데이터 분포 (Histogram)', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
print("✓ [창 1] 주요 변수별 히스토그램 표시")
plt.show()

# 2.4 상관관계 분석 (Heatmap)
print("\n[2.4] 변수 간 상관관계 분석 (Heatmap)")
print("-" * 80)

# 숫자형 데이터만 선택 (문자열은 상관계수 계산 불가)
df_numeric = df.select_dtypes(include=[np.number])

# 피어슨 상관계수(Pearson Correlation) 계산
# - 범위: -1 ~ +1
# - +1: 완벽한 양의 상관 (한 변수 증가 → 다른 변수도 증가)
# - -1: 완벽한 음의 상관 (한 변수 증가 → 다른 변수 감소)
# - 0: 상관관계 없음
# 
# 다중공선성(Multicollinearity) 체크:
# - 독립변수끼리 상관계수가 0.8~0.9 이상이면 문제
# - 예: thinness 5-9 years와 thinness 1-19 years = 0.94
#   → 거의 같은 정보를 담고 있어 하나만 사용해야 함
correlation_matrix = df_numeric.corr()

# Heatmap 그리기
plt.figure(figsize=(16, 14))
# mask: 상삼각 행렬만 숨김 (대칭 행렬이므로 중복 정보 제거)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            annot=True,      # 각 셀에 숫자 표시
            fmt='.2f',       # 소수점 2자리로 표시
            cmap='coolwarm', # 색상: 빨강(양), 파랑(음)
            center=0,        # 0을 중심으로 색상 설정
            square=True,     # 정사각형 셀
            linewidths=0.5,  # 셀 구분선
            cbar_kws={"shrink": 0.8},  # 컬러바 크기
            mask=mask,       # 상삼각 숨김
            vmin=-1, vmax=1) # 값 범위 고정
plt.title('변수 간 상관관계 히트맵 (Heatmap)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
print("✓ [창 2] 상관관계 히트맵 표시")
plt.show()

# Life expectancy와 상관관계가 높은 변수 출력
# 목적: 어떤 변수가 기대수명에 가장 큰 영향을 미치는지 파악
# - 양의 상관: Schooling(교육), Income composition(소득)
# - 음의 상관: Adult Mortality(성인사망률), HIV/AIDS
target_corr = correlation_matrix['Life expectancy '].drop('Life expectancy ').sort_values(ascending=False)
print(f"\nLife Expectancy와 상관관계가 높은 변수 (상위 10개):")
print(target_corr.head(10).to_string())

# ============================================================================
# 3. 데이터 전처리
# ============================================================================
print("\n" + "=" * 80)
print("[3단계] 데이터 전처리 (개선 버전)")
print("=" * 80)

# 3.1 결측치 처리
print("\n[3.1] 결측치 처리")
print("-" * 80)
df_cleaned = df.copy()

# 숫자형 변수의 결측치를 중앙값(median)으로 대체
# 
# 왜 중앙값(median)을 사용하나?
# - 평균(mean)은 이상치에 민감함
# - 중앙값은 이상치에 강건(robust)함
# 
# 예시:
# 데이터: [1, 2, 3, 100]
# - 평균 = 26.5 (100 때문에 왜곡됨)
# - 중앙값 = 2.5 (대표성 있음)
# 
# 다른 방법들:
# - 평균 대체: 정규분포일 때 적합
# - 최빈값 대체: 범주형 데이터에 적합
# - 삭제: 결측치가 매우 적을 때 (< 5%)
# - 예측 모델: KNN, 회귀 등으로 결측치 예측
for col in df_cleaned.select_dtypes(include=[np.number]).columns:
    median_val = df_cleaned[col].median()
    df_cleaned[col].fillna(median_val, inplace=True)

print(f"✓ 숫자형 변수 결측치를 중앙값으로 대체")
print(f"  - 전처리 후 결측치: {df_cleaned.isnull().sum().sum()}개")

# 3.2 범주형 변수 인코딩 (개선: One-Hot Encoding 사용)
print("\n[3.2] 범주형 변수 인코딩 (One-Hot Encoding)")
print("-" * 80)

# ❌ 기존 방식 (LabelEncoder):
# Country를 0, 1, 2로 변환
# → 문제: 모델이 2 > 1 > 0이라는 잘못된 순서 관계 학습
# 
# ✅ 개선 방식 (One-Hot Encoding):
# Country를 여러 개의 이진 변수로 변환
# 예: Korea=[1,0,0], USA=[0,1,0], Japan=[0,0,1]
# → 순서 관계 없이 각 국가를 독립적으로 표현
# 
# drop_first=True:
# - 다중공선성 방지
# - 예: 3개 국가면 2개 변수만 필요
#   (셋 다 0이면 자동으로 첫 번째 국가)
print(f"인코딩 전 컬럼 수: {df_cleaned.shape[1]}")
df_cleaned = pd.get_dummies(df_cleaned, columns=['Country', 'Status'], drop_first=True)
print(f"✓ One-Hot Encoding 완료")
print(f"  - 인코딩 후 컬럼 수: {df_cleaned.shape[1]}")
print(f"  - Country와 Status가 여러 개의 이진 변수로 확장됨")

# 3.3 특성(X)과 타겟(y) 분리
print("\n[3.3] 특성(X)과 타겟(y) 분리")
print("-" * 80)

# X: 독립변수 (입력) - 기대수명을 예측하기 위한 변수들
#    예: 교육 수준, GDP, 성인 사망률 등
# y: 종속변수 (출력) - 예측하고자 하는 기대수명
#    예: 75.3년, 82.1년 등
X = df_cleaned.drop(['Life expectancy '], axis=1)
y = df_cleaned['Life expectancy ']

print(f"✓ 데이터 분리 완료")
print(f"  - 특성(X): {X.shape}")
print(f"  - 타겟(y): {y.shape}")

# 3.4 다중공선성 제거 (개선: VIF 사용)
print("\n[3.4] 다중공선성 제거 (VIF 기반)")
print("-" * 80)

# VIF (Variance Inflation Factor): 분산 팽창 인수
# - 각 변수가 다른 변수들로 얼마나 설명되는지 측정
# - VIF = 1 / (1 - R²)
# 
# 해석:
# - VIF = 1: 다중공선성 없음
# - VIF = 5~10: 중간 정도의 다중공선성
# - VIF > 10: 심각한 다중공선성 → 제거 필요
# 
# 왜 문제인가?
# - 회귀 계수가 불안정해짐 (데이터 조금만 바뀌어도 계수가 크게 변함)
# - 표준 오차가 커져서 통계적 유의성 검정이 어려움
# - 계수 해석이 어려워짐
print("VIF 계산 중... (시간이 소요될 수 있습니다)")

# VIF 계산 함수
def calculate_vif(dataframe):
    """
    데이터프레임의 각 컬럼에 대한 VIF를 계산
    
    Parameters:
    -----------
    dataframe : pandas.DataFrame
        VIF를 계산할 데이터프레임
        
    Returns:
    --------
    pandas.DataFrame
        각 변수와 VIF 값을 담은 데이터프레임
    """
    vif_data = pd.DataFrame()
    vif_data["feature"] = dataframe.columns
    vif_data["VIF"] = [variance_inflation_factor(dataframe.values, i) 
                       for i in range(len(dataframe.columns))]
    return vif_data.sort_values('VIF', ascending=False)

# 초기 VIF 계산
vif_df = calculate_vif(X)
print("\n초기 VIF 값 (상위 10개):")
print(vif_df.head(10).to_string(index=False))

# VIF > 10인 변수 반복 제거
# 알고리즘:
# 1. 가장 높은 VIF 변수 제거
# 2. 다시 VIF 계산
# 3. 모든 VIF < 10이 될 때까지 반복
high_vif_features = []
while True:
    vif_df = calculate_vif(X)
    max_vif = vif_df['VIF'].max()
    
    if max_vif > 10:  # VIF 임계값
        feature_to_drop = vif_df.loc[vif_df['VIF'].idxmax(), 'feature']
        high_vif_features.append((feature_to_drop, max_vif))
        X = X.drop(columns=[feature_to_drop])
        print(f"  제거: {feature_to_drop} (VIF={max_vif:.2f})")
    else:
        break

print(f"\n✓ 다중공선성 제거 완료")
print(f"  - 제거된 변수 수: {len(high_vif_features)}개")
print(f"  - 남은 변수 수: {X.shape[1]}개")
print(f"  - 최대 VIF: {vif_df['VIF'].max():.2f}")

# 3.5 중요 변수 선택 (개선: 상관관계 기반)
print("\n[3.5] 중요 변수 선택 (상관관계 기반)")
print("-" * 80)

# 타겟과의 상관관계가 약한 변수 제거
# - 상관계수 절댓값이 0.1 미만인 변수는 기대수명과 거의 무관
# - 이런 변수는 제거해도 성능에 영향 없고, 모델이 단순해짐
# 
# 장점:
# - 모델 복잡도 감소
# - 과적합 방지
# - 학습 시간 단축
# - 해석이 쉬워짐

# X에 남아있는 숫자형 컬럼만 선택
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
X_numeric = X[numeric_features]

# 상관계수 재계산
temp_df = pd.concat([X_numeric, y], axis=1)
correlation_with_target = temp_df.corr()['Life expectancy '].drop('Life expectancy ')

# 절댓값 기준 정렬
correlation_with_target = correlation_with_target.sort_values(key=abs, ascending=False)

print("\n타겟과의 상관관계 (상위 10개):")
print(correlation_with_target.head(10).to_string())

# 상관관계가 약한 변수 제거 (절댓값 < 0.1)
weak_features = correlation_with_target[abs(correlation_with_target) < 0.1].index.tolist()
if len(weak_features) > 0:
    print(f"\n상관관계가 약한 변수 ({len(weak_features)}개):")
    for feat in weak_features:
        print(f"  - {feat}: {correlation_with_target[feat]:.4f}")
    X = X.drop(columns=weak_features)
else:
    print("\n✓ 모든 변수가 타겟과 유의미한 상관관계를 가짐")

print(f"\n✓ 변수 선택 완료")
print(f"  - 최종 변수 수: {X.shape[1]}개")

# 3.6 학습/테스트 데이터 분할
print("\n[3.6] 학습/테스트 데이터 분할")
print("-" * 80)

# 데이터를 학습용(80%)과 테스트용(20%)으로 분할
# 
# 목적:
# - 학습 데이터: 모델을 학습시키는 데 사용
# - 테스트 데이터: 모델의 일반화 성능 평가 (미래 데이터 성능 추정)
# 
# 왜 분할하나?
# - 학습 데이터로만 평가하면 과적합 여부를 알 수 없음
# - 테스트 데이터는 모델이 "처음 보는" 데이터
# - 실제 운영 환경에서의 성능을 추정 가능
# 
# random_state=42: 재현 가능성
# - 시드를 고정하면 항상 같은 방식으로 분할됨
# - 실험 결과를 재현하거나 비교할 때 필수
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"✓ 데이터 분할 완료 (Train: 80%, Test: 20%)")
print(f"  - 학습 데이터: {X_train.shape}")
print(f"  - 테스트 데이터: {X_test.shape}")

# 3.7 데이터 스케일링
print("\n[3.7] 데이터 스케일링 (StandardScaler)")
print("-" * 80)

# StandardScaler: 평균=0, 표준편차=1로 정규화
# 
# 변환 공식: z = (x - μ) / σ
# - μ: 평균 (mean)
# - σ: 표준편차 (standard deviation)
# 
# 예시:
# 원본: [10, 20, 30]
# 평균: 20, 표준편차: 8.16
# 변환: [(10-20)/8.16, (20-20)/8.16, (30-20)/8.16]
#     = [-1.22, 0, 1.22]
# 
# 왜 필요한가?
# 1. 변수들의 단위가 다름
#    - GDP: 1,000 ~ 100,000 (달러)
#    - 사망률: 0 ~ 1 (비율)
#    → GDP가 스케일이 크므로 모델이 GDP에 편향될 수 있음
# 
# 2. 수치 안정성
#    - 매우 큰 값이나 작은 값은 계산 오류 발생 가능
# 
# 3. 알고리즘 성능
#    - 경사하강법 기반 알고리즘은 스케일링 시 빠르게 수렴
# 
# ⚠️ 중요: fit_transform vs transform
# - fit_transform: 통계량 계산 + 변환 (학습 데이터에만!)
# - transform: 기존 통계량으로 변환 (테스트 데이터에!)
# - 테스트 데이터로 fit하면 Data Leakage 발생!
#   (테스트 정보가 학습에 유출되어 성능이 부풀려짐)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # 학습: 평균/표준편차 계산 + 변환
X_test_scaled = scaler.transform(X_test)        # 테스트: 학습 데이터의 평균/표준편차로 변환

# DataFrame으로 변환 (컬럼명과 인덱스 유지 - 가독성과 디버깅 용이)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print(f"✓ StandardScaler를 사용한 정규화 완료")
print(f"  - 변환 후 평균: 0 (정확히는 ~1e-16)")
print(f"  - 변환 후 표준편차: 1")
print(f"\n스케일링 예시 (첫 번째 변수):")
feature_name = X_train.columns[0]
print(f"  - 변수명: {feature_name}")
print(f"  - 원본 범위: [{X_train[feature_name].min():.2f}, {X_train[feature_name].max():.2f}]")
print(f"  - 변환 후 범위: [{X_train_scaled[feature_name].min():.2f}, {X_train_scaled[feature_name].max():.2f}]")

# ============================================================================
# 4. 모델링 - Linear Regression
# ============================================================================
print("\n" + "=" * 80)
print("[4단계] 모델링 - Linear Regression")
print("=" * 80)

print("\n[4.1] 모델 학습")
print("-" * 80)

# Linear Regression: 선형 회귀 모델
# 
# 수학적 모델:
# y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε
# 
# - y: 타겟 변수 (Life Expectancy)
# - β₀: 절편 (intercept) - 모든 x가 0일 때 y값
# - β₁, β₂, ..., βₙ: 회귀 계수 (coefficients) - 각 변수의 영향력
# - x₁, x₂, ..., xₙ: 독립 변수들
# - ε: 오차 (error)
# 
# 학습 방법: 최소제곱법 (Ordinary Least Squares, OLS)
# - 목표: Σ(실제값 - 예측값)² 최소화
# - 즉, 모든 데이터 포인트와 회귀선 사이의 거리 제곱 합을 최소화
# 
# 예시:
# Life Expectancy = 50 + 2.5(Schooling) - 0.3(Adult Mortality) + ...
# → 교육 연수가 1년 증가하면 기대수명이 2.5년 증가
# → 성인 사망률이 1% 증가하면 기대수명이 0.3년 감소
# 
# 가정 (위배 시 성능 저하):
# 1. 선형성: X와 y가 선형 관계
# 2. 독립성: 잔차들이 서로 독립
# 3. 등분산성: 잔차의 분산이 일정
# 4. 정규성: 잔차가 정규분포
# 5. 다중공선성 없음: X들끼리 독립 (우리가 해결함!)
model = LinearRegression()
model.fit(X_train_scaled, y_train)

print("✓ Linear Regression 모델 학습 완료!")
print(f"  - 학습된 계수(coefficient) 수: {len(model.coef_)}")
print(f"  - 절편(intercept): {model.intercept_:.4f}")

# ============================================================================
# 5. 예측 및 성능 평가
# ============================================================================
print("\n" + "=" * 80)
print("[5단계] 예측 및 성능 평가")
print("=" * 80)

# 5.1 예측
# 학습 데이터와 테스트 데이터 모두 예측
# - 학습 데이터 예측: 과적합(overfitting) 확인용
# - 테스트 데이터 예측: 실제 성능 평가용
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# 5.2 성능 지표 계산
# 
# MSE (Mean Squared Error): 평균 제곱 오차
# - 공식: Σ(실제 - 예측)² / n
# - 오차를 제곱하므로 큰 오차에 더 큰 페널티
# - 작을수록 좋음
# - 단점: 단위가 제곱되어 해석 어려움
# 
# RMSE (Root Mean Squared Error): MSE의 제곱근
# - 공식: √MSE
# - 원래 단위로 해석 가능
# - 예: RMSE=3.5 → 평균적으로 3.5년 오차
# - 가장 널리 사용되는 지표
# 
# MAE (Mean Absolute Error): 평균 절대 오차
# - 공식: Σ|실제 - 예측| / n
# - 모든 오차를 동등하게 취급
# - 이상치에 덜 민감 (RMSE보다)
# - 해석이 직관적
# 
# R² Score (결정계수): 모델의 설명력
# - 공식: 1 - (SS_res / SS_tot)
# - 범위: -∞ ~ 1 (일반적으로 0 ~ 1)
# - 의미: 모델이 데이터 변동의 몇 %를 설명하는가
# - 예: R²=0.85 → 85% 설명 가능
# - 해석:
#   * 0.7 ~ 0.8: 양호
#   * 0.8 ~ 0.9: 좋음
#   * 0.9 이상: 매우 좋음 (또는 과적합 의심)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("\n[5.1] 모델 성능 평가")
print("=" * 80)
print(f"{'평가 지표':<20} {'학습 데이터':<20} {'테스트 데이터':<20}")
print("-" * 80)
print(f"{'MSE':<20} {train_mse:<20.4f} {test_mse:<20.4f}")
print(f"{'RMSE':<20} {train_rmse:<20.4f} {test_rmse:<20.4f}")
print(f"{'MAE':<20} {train_mae:<20.4f} {test_mae:<20.4f}")
print(f"{'R² Score':<20} {train_r2:<20.4f} {test_r2:<20.4f}")

# 과적합(Overfitting) 진단
# - Train 성능 >> Test 성능이면 과적합
# - 예: Train R²=0.95, Test R²=0.70 → 차이 0.25 (과적합!)
# - 일반적으로 차이가 0.05 이하면 적절함
r2_diff = train_r2 - test_r2
print(f"\n[과적합 진단]")
print(f"  - R² 차이: {r2_diff:.4f}")
if r2_diff < 0.05:
    print(f"  - 판정: ✅ 과적합 없음 (차이 < 0.05)")
elif r2_diff < 0.10:
    print(f"  - 판정: ⚠️  약간의 과적합 (0.05 ≤ 차이 < 0.10)")
else:
    print(f"  - 판정: ❌ 과적합 발생 (차이 ≥ 0.10)")
    print(f"  - 해결 방법: 정규화(Ridge/Lasso), 변수 추가 제거, 더 많은 데이터")

# 5.3 회귀 계수 분석
print("\n[5.2] 회귀 계수 분석 (상위 15개)")
print("=" * 80)

# 회귀 계수(Coefficient) 해석
# - 양수: 해당 변수 증가 → 기대수명 증가
# - 음수: 해당 변수 증가 → 기대수명 감소
# - 절댓값 클수록: 영향력이 큼
# 
# ⚠️ 주의사항:
# 1. 스케일링 후의 계수이므로 원래 단위가 아님
#    - 모든 변수가 같은 스케일(평균=0, 표준편차=1)
#    - 따라서 계수의 크기를 직접 비교 가능
# 
# 2. 다른 변수들을 고정했을 때의 효과
#    - "다른 모든 변수가 같을 때" 해당 변수만의 영향
# 
# 3. 인과관계가 아닌 상관관계
#    - 계수가 크다고 해서 인과관계는 아님
#    - 숨겨진 제3의 변수가 있을 수 있음
coefficients = pd.DataFrame({
    '특성': X_train_scaled.columns,
    '계수': model.coef_
}).sort_values('계수', key=abs, ascending=False)  # 절댓값 기준 내림차순

print(coefficients.head(15).to_string(index=False))

print("\n[계수 해석 예시]")
top_positive = coefficients[coefficients['계수'] > 0].head(3)
top_negative = coefficients[coefficients['계수'] < 0].head(3)
print(f"\n가장 긍정적 영향을 미치는 변수 (상위 3개):")
for idx, row in top_positive.iterrows():
    print(f"  - {row['특성']}: {row['계수']:.4f}")
    print(f"    → 이 변수가 1 표준편차 증가하면 기대수명이 약 {abs(row['계수']):.2f}년 증가")

print(f"\n가장 부정적 영향을 미치는 변수 (상위 3개):")
for idx, row in top_negative.iterrows():
    print(f"  - {row['특성']}: {row['계수']:.4f}")
    print(f"    → 이 변수가 1 표준편차 증가하면 기대수명이 약 {abs(row['계수']):.2f}년 감소")

# ============================================================================
# 6. 시각화
# ============================================================================
print("\n" + "=" * 80)
print("[6단계] 예측 결과 시각화")
print("=" * 80)

# 6.1 실제값 vs 예측값 & 잔차 플롯
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 좌측: 실제값 vs 예측값 산점도
# 이상적인 모델:
# - 모든 점이 빨간 대각선(y=x) 위에 위치
# - 선에서 멀수록 예측 오차가 큼
# 
# 패턴 분석:
# - 점들이 선 위쪽에 몰림: 과소 예측 (실제보다 낮게 예측)
# - 점들이 선 아래쪽에 몰림: 과대 예측 (실제보다 높게 예측)
# - 특정 범위에서 산개: 그 범위에서 예측이 불안정
axes[0].scatter(y_test, y_test_pred, alpha=0.6, edgecolors='k', linewidth=0.5, s=30)
axes[0].plot([y_test.min(), y_test.max()], 
             [y_test.min(), y_test.max()], 
             'r--', lw=2, label='완벽한 예측선 (y=x)')
axes[0].set_xlabel('실제 Life Expectancy (년)', fontsize=11)
axes[0].set_ylabel('예측 Life Expectancy (년)', fontsize=11)
axes[0].set_title(f'예측 결과 산점도\nR² = {test_r2:.4f}, RMSE = {test_rmse:.4f}', 
                  fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 우측: 잔차 플롯 (Residual Plot)
# 잔차 = 실제값 - 예측값
# 
# 좋은 모델의 잔차 패턴:
# ✅ 0을 중심으로 무작위로 분포 (랜덤)
# ✅ 패턴이 없어야 함 (No Pattern)
# ✅ 등분산성: 퍼짐 정도가 일정
# 
# 문제 있는 패턴:
# ❌ U자 또는 곡선 형태: 비선형 관계 존재
#    → 해결: 다항 회귀, 비선형 모델 사용
# 
# ❌ 퍼짐이 점점 커짐 (깔때기 모양): 등분산성 위배
#    → 해결: 로그 변환, 가중 회귀
# 
# ❌ 특정 영역에 몰림: 이상치 존재
#    → 해결: 이상치 제거 또는 robust 회귀
residuals = y_test - y_test_pred
axes[1].scatter(y_test_pred, residuals, alpha=0.6, edgecolors='k', linewidth=0.5, s=30)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2, label='완벽한 예측 (잔차=0)')
axes[1].set_xlabel('예측 Life Expectancy (년)', fontsize=11)
axes[1].set_ylabel('잔차 (실제값 - 예측값)', fontsize=11)
axes[1].set_title('잔차 플롯 (Residual Plot)', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
print("✓ [창 3] 예측 결과 및 잔차 플롯 표시")
plt.show()

# 6.2 회귀 계수 시각화
plt.figure(figsize=(10, 8))
top_coef = coefficients.head(15)
# 양수=파랑(긍정적 영향), 음수=빨강(부정적 영향)으로 구분
colors = ['red' if x < 0 else 'blue' for x in top_coef['계수']]
plt.barh(top_coef['특성'], top_coef['계수'], color=colors, alpha=0.7, edgecolor='black')
plt.xlabel('회귀 계수 값 (스케일링 후)', fontsize=11)
plt.ylabel('특성', fontsize=11)
plt.title('상위 15개 특성의 회귀 계수\n파랑: 긍정적 영향 | 빨강: 부정적 영향', 
          fontsize=12, fontweight='bold')
plt.axvline(x=0, color='black', linestyle='-', linewidth=1)
plt.gca().invert_yaxis()  # 가장 중요한 변수가 위로
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
print("✓ [창 4] 회귀 계수 시각화 표시")
plt.show()

# 6.3 실제 데이터와 예측 데이터 분포 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 학습 데이터 분포
# - 파랑(실제)과 빨강(예측)이 비슷하면 모델이 데이터 분포를 잘 학습
# - 차이가 크면 모델이 특정 범위를 과소/과대 예측
axes[0].hist(y_train, bins=30, alpha=0.6, color='blue', edgecolor='black', label='실제값 (Train)')
axes[0].hist(y_train_pred, bins=30, alpha=0.6, color='red', edgecolor='black', label='예측값 (Train)')
axes[0].set_xlabel('Life Expectancy (년)', fontsize=11)
axes[0].set_ylabel('빈도', fontsize=11)
axes[0].set_title('학습 데이터 분포 비교', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 테스트 데이터 분포
axes[1].hist(y_test, bins=30, alpha=0.6, color='blue', edgecolor='black', label='실제값 (Test)')
axes[1].hist(y_test_pred, bins=30, alpha=0.6, color='red', edgecolor='black', label='예측값 (Test)')
axes[1].set_xlabel('Life Expectancy (년)', fontsize=11)
axes[1].set_ylabel('빈도', fontsize=11)
axes[1].set_title('테스트 데이터 분포 비교', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
print("✓ [창 5] 실제 데이터와 예측 데이터 분포 비교")
plt.show()

# 6.4 최종 VIF 확인
print("\n[6.4] 최종 VIF 확인")
print("-" * 80)
final_vif = calculate_vif(X_train_scaled)
print("\n최종 VIF 값 (상위 10개):")
print(final_vif.head(10).to_string(index=False))
print(f"\n✓ 모든 VIF < 10 확인 완료 (다중공선성 해결)")

# ============================================================================
# 7. 결과 저장
# ============================================================================
print("\n" + "=" * 80)
print("[7단계] 결과 저장")
print("=" * 80)

# 성능 지표를 CSV로 저장 (나중에 다른 모델과 비교용)
results = {
    '모델': 'Linear Regression (개선)',
    'Test_MSE': test_mse,
    'Test_RMSE': test_rmse,
    'Test_MAE': test_mae,
    'Test_R2': test_r2,
    'Train_MSE': train_mse,
    'Train_RMSE': train_rmse,
    'Train_MAE': train_mae,
    'Train_R2': train_r2,
    'R2_Difference': r2_diff,
    'Features_Used': X_train_scaled.shape[1],
    'Max_VIF': final_vif['VIF'].max()
}

results_df = pd.DataFrame([results])
results_df.to_csv('linear_regression_metrics_improved.csv', index=False)
print("✓ 성능 지표 저장: linear_regression_metrics_improved.csv")

# 회귀 계수 저장 (어떤 변수가 중요한지 분석용)
coefficients.to_csv('linear_regression_coefficients_improved.csv', index=False)
print("✓ 회귀 계수 저장: linear_regression_coefficients_improved.csv")

# 제거된 변수 목록 저장
if len(high_vif_features) > 0:
    removed_features_df = pd.DataFrame(high_vif_features, columns=['Feature', 'VIF'])
    removed_features_df.to_csv('removed_high_vif_features.csv', index=False)
    print("✓ 제거된 변수 목록 저장: removed_high_vif_features.csv")

# 최종 VIF 저장
final_vif.to_csv('final_vif_values.csv', index=False)
print("✓ 최종 VIF 값 저장: final_vif_values.csv")

# ============================================================================
# 최종 요약
# ============================================================================
print("\n" + "=" * 80)
print("LINEAR REGRESSION 분석 완료! (개선 버전)")
print("=" * 80)

print(f"\n📊 최종 성능 요약:")
print(f"  - R² Score: {test_r2:.4f} ({test_r2*100:.2f}% 설명력)")
print(f"  - RMSE: {test_rmse:.4f}년 (평균 오차)")
print(f"  - MAE: {test_mae:.4f}년 (절대 오차)")

print(f"\n🔧 전처리 요약

SyntaxError: unterminated f-string literal (detected at line 749) (64025470.py, line 749)